# Peephole LSTM Test & Performance Comparison (Speed & Memory)

* [Imports](#Importing-necessary-modules)
* [Load & Definition](#Loading-and-defining-modules)
    * [Autograd Functions](#Autograd-Functions)
    * [Module Classes](#Module-classes-(C++,-CUDA,-PyTorch))
* [Models](#Defining-models)
    * [Definition](#Definition)
    * [Instantiation](#Instantiation)
    * [Parameter Synchronization](#Parameter-Synchronization)
* [Fake Dataset](#Creating-a-fake-dataset)
* [Sanity Check](#Sanity-check:-output-comparison)
    * [Forward Outputs](#Forward-Outputs)
    * [Backward Gradients](#Backward-Gradients)
* [Forward Performance](#Forward-time-comparison)
* [+Backward Performance](#+Backward-time-comparison)

---

## Importing necessary modules
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

In [1]:
import torch
from torch import nn
from torch.utils.cpp_extension import load
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader

import math
from collections import OrderedDict
from time import sleep

torch.__version__

'0.4.1'

---

## Loading and defining modules
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

### Autograd Functions
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

In [2]:
_bn_peephole_lstm_layer_cpp = load('ln_peephole_lstm_layer', ['./ln_peephole_lstm_layer.cpp'])

########################################################################################################################

class LNPeepholeLSTMFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input, weight_ih, weight_hh, weight_ch, bias, gamma_ih, gamma_hh, gamma_ch, gamma_tanh_cell, beta_tanh_cell, old_h, old_cell, epsilon, dropout_p, training):
        
        outputs = _bn_peephole_lstm_layer_cpp.forward(input, weight_ih, weight_hh, weight_ch, bias,
                                                    gamma_ih, gamma_hh, gamma_ch, gamma_tanh_cell, beta_tanh_cell,
                                                    old_h, old_cell,
                                                    epsilon, dropout_p, training)
        
        out, new_h, new_cell = outputs[:3]
        
        variables = outputs[3:] + [weight_ih, weight_hh, weight_ch,
                                    gamma_ih, gamma_hh, gamma_ch, gamma_tanh_cell]
        ctx.training = training # Boolean value stored this way since only tensors can be stored using save_for_backward
        ctx.save_for_backward(*variables)

        return out, new_h, new_cell

    @staticmethod
    def backward(ctx, grad_output, grad_h, grad_cell):
        outputs = _bn_peephole_lstm_layer_cpp.backward(
            grad_output.contiguous(), grad_h.contiguous(), grad_cell.contiguous(), *ctx.saved_tensors, ctx.training)
        
        (d_old_h, d_old_cell, d_input,
         d_weight_ih, d_weight_hh, d_weight_ch, d_bias,
         d_gamma_ih, d_gamma_hh, d_gamma_ch, d_gamma_tanh_cell, d_beta_tanh_cell) = outputs
        
        return (d_input, d_weight_ih, d_weight_hh, d_weight_ch, d_bias,
                d_gamma_ih, d_gamma_hh, d_gamma_ch, d_gamma_tanh_cell, d_beta_tanh_cell,
                d_old_h, d_old_cell,
                None, None, None)

c:\users\0107w\anaconda3\envs\pytorch\lib\site-packages\torch\utils\cpp_extension.py:92: UserWarning: Error checking compiler version: Command '['c++']' returned non-zero exit status 1.
  warnings.warn('Error checking compiler version: {}'.format(error))
c:\users\0107w\anaconda3\envs\pytorch\lib\site-packages\torch\utils\cpp_extension.py:118: UserWarning: 

                               !! WARNING !!

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Your compiler (c++) may be ABI-incompatible with PyTorch!
Please use a compiler that is ABI-compatible with GCC 4.9 and above.
See https://gcc.gnu.org/onlinedocs/libstdc++/manual/abi.html.

See https://gist.github.com/goldsborough/d466f43e8ffc948ff92de7486c5216d6
for instructions on how to install GCC 4.9 or higher.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

                              !! WARNING !!

  warnings.warn(ABI_INCOMPATIBILITY_WARNING.format(compiler))


### Module classes (C++, CUDA, PyTorch)
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

In [3]:
class LNPeepholeLSTMTorch(nn.Module):
    def __init__(self, input_size, hidden_size, dropout=0., eps=1e-05):
        if not 0 <= dropout <= 1:
            raise ValueError(f"Invalid dropout value : {dropout} dropout must be in range [0, 1].")
        super(LNPeepholeLSTMTorch, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.dropout = float(dropout)
        self.eps = eps
        
        self.register_parameter('weight_ih', nn.Parameter(torch.empty(4 * hidden_size, input_size)))
        self.register_parameter('weight_hh', nn.Parameter(torch.empty(4 * hidden_size, hidden_size)))
        self.register_parameter('weight_ch', nn.Parameter(torch.empty(3 * hidden_size, hidden_size)))
        self.register_parameter('bias', nn.Parameter(torch.empty(4 * hidden_size)))
        
        self.register_parameter('gamma_ih', nn.Parameter(torch.empty(4 * hidden_size)))
        self.register_parameter('gamma_hh', nn.Parameter(torch.empty(4 * hidden_size)))
        self.register_parameter('gamma_ch', nn.Parameter(torch.empty(3 * hidden_size)))
        self.register_parameter('gamma_tanh_cell', nn.Parameter(torch.empty(hidden_size)))
        self.register_parameter('beta_tanh_cell', nn.Parameter(torch.empty(hidden_size)))
        
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1.0 / math.sqrt(self.input_size + 2 * self.hidden_size)
        self.weight_ih.data.uniform_(-stdv, +stdv)
        self.weight_hh.data.uniform_(-stdv, +stdv)
        self.weight_ch.data.uniform_(-stdv, +stdv)
        
        self.bias.data.zero_()
        self.bias.data[:self.hidden_size].fill_(1.)
        
        self.gamma_ih.data.uniform_()
        self.gamma_hh.data.uniform_()
        self.gamma_ch.data.uniform_()
        self.gamma_tanh_cell.data.uniform_()
        self.beta_tanh_cell.data.zero_()
    
    def forward(self, input, states):
        assert input.dim() == 3
        outputs = input.new_empty((input.size(0), input.size(1), self.hidden_size))
        
        h = states[0].clone()
        c = states[1].clone()
        
        weight_ih = self.weight_ih.t()
        weight_hh = self.weight_hh.t()
        weight_ch = self.weight_ch.t()
        
        ih = torch.matmul(input.transpose(0, 1), weight_ih)
        
        for i in range(input.size(1)):
            h = F.dropout(h, p=self.dropout, training=self.training)
            
            gates = (F.layer_norm(ih[i], (4 * self.hidden_size,), weight=self.gamma_ih, bias=None, eps=self.eps)
                     + F.layer_norm(torch.mm(h, weight_hh), (4 * self.hidden_size,), weight=self.gamma_hh, bias=None, eps=self.eps)
                     + self.bias)
            
            gates = torch.cat((gates[:, :3 * self.hidden_size].add(F.layer_norm(torch.mm(c, weight_ch), (3 * self.hidden_size,), weight=self.gamma_ch, bias=None, eps=self.eps)).sigmoid(), gates[:, 3 * self.hidden_size:].tanh()), dim=1).chunk(chunks=4, dim=1)
            
            c = torch.addcmul(gates[1] * gates[3], c, gates[0])
            h = gates[2] * F.layer_norm(c.tanh(), (self.hidden_size,), weight=self.gamma_tanh_cell, bias=self.beta_tanh_cell, eps=self.eps)
            
            outputs[:, i] = h
        
        outputs = torch.nn.functional.dropout(outputs, p=self.dropout, training=self.training)
    
        return outputs, (h, c)
    
    def __repr__(self):
        return f"LNPeepholeLSTMTorch(input_size={self.input_size}, hidden_size={self.hidden_size}, dropout={self.dropout}, momentum={self.momentum}, eps={self.eps})"

########################################################################################################################

class LNPeepholeLSTMCPP(nn.Module):
    def __init__(self, input_size, hidden_size, dropout=0., eps=1e-05):
        if not 0 <= dropout <= 1:
            raise ValueError(f"Invalid dropout value : {dropout} dropout must be in range [0, 1].")
            
        super(LNPeepholeLSTMCPP, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.dropout = float(dropout)
        self.eps = eps
        
        self.register_parameter('weight_ih', nn.Parameter(torch.empty(4 * hidden_size, input_size)))
        self.register_parameter('weight_hh', nn.Parameter(torch.empty(4 * hidden_size, hidden_size)))
        self.register_parameter('weight_ch', nn.Parameter(torch.empty(3 * hidden_size, hidden_size)))
        self.register_parameter('bias', nn.Parameter(torch.empty(4 * hidden_size)))
        
        self.register_parameter('gamma_ih', nn.Parameter(torch.empty(4 * hidden_size)))
        self.register_parameter('gamma_hh', nn.Parameter(torch.empty(4 * hidden_size)))
        self.register_parameter('gamma_ch', nn.Parameter(torch.empty(3 * hidden_size)))
        self.register_parameter('gamma_tanh_cell', nn.Parameter(torch.empty(hidden_size)))
        self.register_parameter('beta_tanh_cell', nn.Parameter(torch.empty(hidden_size)))
        
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1.0 / math.sqrt(self.input_size + 2 * self.hidden_size)
        self.weight_ih.data.uniform_(-stdv, +stdv)
        self.weight_hh.data.uniform_(-stdv, +stdv)
        self.weight_ch.data.uniform_(-stdv, +stdv)
        
        self.bias.data.zero_()
        self.bias.data[:self.hidden_size].fill_(1.)
        
        self.gamma_ih.data.uniform_()
        self.gamma_hh.data.uniform_()
        self.gamma_ch.data.uniform_()
        self.gamma_tanh_cell.data.uniform_()
        self.beta_tanh_cell.data.zero_()
        
    def forward(self, input, state):
        input = input.transpose(0, 1).contiguous()
        
        output, new_h, new_cell = LNPeepholeLSTMFunction.apply(input, self.weight_ih, self.weight_hh, self.weight_ch, self.bias, self.gamma_ih, self.gamma_hh, self.gamma_ch, self.gamma_tanh_cell, self.beta_tanh_cell, state[0], state[1], self.eps, self.dropout, self.training)
        
        return output.transpose(0, 1).contiguous(), (new_h, new_cell)
    
    def __repr__(self):
        return f"LNPeepholeLSTMCPP(input_size={self.input_size}, hidden_size={self.hidden_size}, dropout={self.dropout}, momentum={self.momentum}, eps={self.eps})"

---

## Defining models
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

### Definition
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

In [4]:
class LNPeepholeTorch(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout=0, eps=1e-05):
        super().__init__()
        self.lstm0 = LNPeepholeLSTMTorch(input_size, hidden_size, dropout, eps)
        self.lstm1 = LNPeepholeLSTMTorch(hidden_size, hidden_size, dropout, eps)
        self.lstm2 = LNPeepholeLSTMTorch(hidden_size, hidden_size, dropout, eps)
        self.fc = nn.Linear(hidden_size, output_size)
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
    def forward(self, x, states):
        hc1, hc2, hc3 = ((states[0][i], states[1][i]) for i in range(states[0].size(0)))
        x, hc1 = self.lstm0(x, hc1)
        x, hc2 = self.lstm1(x, hc2)
        x, hc3 = self.lstm2(x, hc3)
        x = self.fc(x)
        new_states = (torch.cat((hc1[0], hc2[0], hc3[0])), torch.cat((hc1[1], hc2[1], hc3[1])))
        return x, new_states

########################################################################################################################
    
class LNPeepholeCPP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout=0, eps=1e-05):
        super().__init__()
        self.lstm0 = LNPeepholeLSTMCPP(input_size, hidden_size, dropout, eps)
        self.lstm1 = LNPeepholeLSTMCPP(hidden_size, hidden_size, dropout, eps)
        self.lstm2 = LNPeepholeLSTMCPP(hidden_size, hidden_size, dropout, eps)
        self.fc = nn.Linear(hidden_size, output_size)
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
    def forward(self, x, states):
        hc1, hc2, hc3 = ((states[0][i], states[1][i]) for i in range(states[0].size(0)))
        x, hc1 = self.lstm0(x, hc1)
        x, hc2 = self.lstm1(x, hc2)
        x, hc3 = self.lstm2(x, hc3)
        x = self.fc(x)
        new_states = (torch.cat((hc1[0], hc2[0], hc3[0])), torch.cat((hc1[1], hc2[1], hc3[1])))
        return x, new_states

### Instantiation
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

In [5]:
device = ('cpu', 'cuda')[1]

input_size = 5
hidden_size = 20
output_size = 10
dropout = 0.
eps = 1e-05

model_torch = LNPeepholeTorch(input_size, hidden_size, output_size, dropout, eps)
model_cpp = LNPeepholeCPP(input_size, hidden_size, output_size, dropout, eps)

model_torch.to(device)
model_cpp.to(device)

models = (model_torch, model_cpp)

### Parameter Synchronization
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

In [6]:
named_parameter_dicts = [
    dict(model_torch.named_parameters()),
    dict(model_cpp.named_parameters())
]

print("Synchronized Parameters:\n")
for common_param_name in set.intersection(*(set(npd.keys()) for npd in named_parameter_dicts)):
    print("\t{}".format(common_param_name))
    for i in range(1, len(named_parameter_dicts)):
        if named_parameter_dicts[i][common_param_name].size() == named_parameter_dicts[0][common_param_name].size():
            named_parameter_dicts[i][common_param_name].data = named_parameter_dicts[0][common_param_name].data
        else:
            raise RuntimeError("Size mismatch\n0:{}\n{i}:{}".format(named_parameter_dicts[0][common_param_name].size(),
                                                                    named_parameter_dicts[i][common_param_name].size()))
print()
print("Exclusive Parameters (Not Synchronized):\n")
for exclusive_param_name in set.union(*(set(npd.keys()) for npd in named_parameter_dicts)) - set.intersection(*(set(npd.keys()) for npd in named_parameter_dicts)):
    print("\t{}".format(exclusive_param_name))

Synchronized Parameters:

	lstm2.gamma_hh
	lstm1.gamma_hh
	lstm2.weight_ih
	fc.bias
	lstm1.gamma_ih
	lstm0.beta_tanh_cell
	lstm2.bias
	lstm1.bias
	lstm1.gamma_ch
	lstm0.weight_hh
	lstm1.weight_ih
	lstm0.gamma_ih
	lstm1.weight_hh
	lstm2.gamma_ih
	lstm1.gamma_tanh_cell
	lstm2.gamma_ch
	lstm0.gamma_tanh_cell
	fc.weight
	lstm2.weight_hh
	lstm0.gamma_hh
	lstm0.bias
	lstm1.weight_ch
	lstm2.weight_ch
	lstm0.weight_ch
	lstm2.gamma_tanh_cell
	lstm0.gamma_ch
	lstm2.beta_tanh_cell
	lstm0.weight_ih
	lstm1.beta_tanh_cell

Exclusive Parameters (Not Synchronized):



---

## Creating a fake dataset
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

In [7]:
def create_fake_loader(dataset_size, sequence_length, batch_size, drop_last=True):
    fake_inputs = torch.randn(dataset_size, sequence_length, input_size)
    fake_targets = torch.randint(high=output_size, size=(dataset_size, sequence_length), dtype=torch.int64)

    fake_dataset = TensorDataset(fake_inputs, fake_targets)

    fake_loader = DataLoader(fake_dataset, batch_size=batch_size, drop_last=drop_last)
    
    return fake_loader

In [8]:
dataset_size = 1000
sequence_length = 20
batch_size = 32

fake_loader = create_fake_loader(dataset_size, sequence_length, batch_size)
print(next(iter(fake_loader))[0].size(), next(iter(fake_loader))[1].size())

torch.Size([32, 20, 5]) torch.Size([32, 20])


---

## Sanity check: output comparison
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

### Forward Outputs
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

In [9]:
hidden = (torch.zeros(3, batch_size, hidden_size, device=device), torch.zeros(3, batch_size, hidden_size, device=device))

inputs = next(iter(fake_loader))[0].to(device)

for model in models:
    model.train()
#     model.eval()

with torch.no_grad():
    print("[model_torch]")
    print("\n{partial output}")
    out0 = model_torch(inputs, hidden)[0]
    print(out0[:4, -2:, :7])
    print("\n")
    print("[model_cpp]")
    print("\n{partial output}")
    out1 = model_cpp(inputs, hidden)[0]
    print(out1[:4, -2:, :7])
    print()
    print(out0.sub(out1).abs().sum())

[model_torch]

{partial output}
tensor([[[ 0.0304,  0.2172, -0.0624, -0.1318, -0.0283,  0.1379, -0.1804],
         [ 0.0303,  0.2071, -0.0516, -0.1227, -0.0175,  0.1362, -0.1736]],

        [[ 0.2551, -0.1635,  0.1305, -0.0884, -0.1512, -0.0729, -0.3870],
         [ 0.2448, -0.1478,  0.1165, -0.1006, -0.1696, -0.0577, -0.3851]],

        [[ 0.4270,  0.0743, -0.3394, -0.3259, -0.0720, -0.1036, -0.2924],
         [ 0.4107,  0.1626, -0.3268, -0.3683, -0.1054, -0.0973, -0.2562]],

        [[ 0.1277,  0.0291, -0.2125, -0.0557,  0.0559,  0.0716, -0.1245],
         [ 0.1189,  0.0394, -0.2497, -0.0533,  0.0603,  0.0882, -0.1024]]],
       device='cuda:0')


[model_cpp]

{partial output}
tensor([[[ 0.0304,  0.2172, -0.0624, -0.1318, -0.0283,  0.1379, -0.1804],
         [ 0.0303,  0.2071, -0.0516, -0.1227, -0.0175,  0.1362, -0.1736]],

        [[ 0.2551, -0.1635,  0.1305, -0.0884, -0.1512, -0.0729, -0.3870],
         [ 0.2448, -0.1478,  0.1165, -0.1006, -0.1696, -0.0577, -0.3851]],

        [[ 0

### Backward Gradients
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

In [10]:
criterion = nn.CrossEntropyLoss()

hidden = (torch.zeros(3, batch_size, hidden_size, device=device), torch.zeros(3, batch_size, hidden_size, device=device))

inputs, targets = next(iter(fake_loader))
inputs = inputs.to(device)
targets = targets.to(device)

for model in models:
    model.train()
#     model.eval()
    model.zero_grad()
    loss = criterion(model(inputs, hidden)[0].flatten(0, 1), targets.flatten(0, 1))
    print(loss)
    loss.backward()

print()
print("model_torch")
# print(model_torch.lstm2.weight_ih.grad[:4, :7])
print(model_torch.lstm2.beta_tanh_cell.grad[:7])
print(model_torch.lstm2.bias.grad[:25])
print("\n")
print("model_cpp")
# print(model_cpp.lstm2.weight_ih.grad[:4, :7])
print(model_cpp.lstm2.beta_tanh_cell.grad[:7])
print(model_torch.lstm2.bias.grad[:25])
print("\n")
print(model_torch.lstm2.weight_ih.grad.sub(model_cpp.lstm2.weight_ih.grad).abs().sum())

tensor(2.3261, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.3261, device='cuda:0', grad_fn=<NllLossBackward>)

model_torch
tensor([ 0.0022, -0.0060,  0.0009,  0.0010, -0.0003,  0.0085, -0.0020],
       device='cuda:0')
tensor([-0.0004, -0.0006, -0.0006, -0.0001, -0.0002, -0.0012,  0.0003, -0.0003,
         0.0008,  0.0017, -0.0014, -0.0005, -0.0004,  0.0020,  0.0034,  0.0001,
        -0.0011, -0.0011,  0.0007,  0.0003, -0.0003, -0.0001, -0.0007, -0.0000,
        -0.0019], device='cuda:0')


model_cpp
tensor([ 0.0022, -0.0060,  0.0009,  0.0010, -0.0003,  0.0085, -0.0020],
       device='cuda:0')
tensor([-0.0004, -0.0006, -0.0006, -0.0001, -0.0002, -0.0012,  0.0003, -0.0003,
         0.0008,  0.0017, -0.0014, -0.0005, -0.0004,  0.0020,  0.0034,  0.0001,
        -0.0011, -0.0011,  0.0007,  0.0003, -0.0003, -0.0001, -0.0007, -0.0000,
        -0.0019], device='cuda:0')


tensor(0.0000, device='cuda:0')


---

## Forward time comparison
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

In [11]:
dataset_size = 1000
sequence_length = 20
batch_size = 32

model_torch.train()
model_cpp.train()

fake_loader = create_fake_loader(dataset_size, sequence_length, batch_size, drop_last=True)

hidden = (torch.zeros(3, batch_size, hidden_size, device=device), torch.zeros(3, batch_size, hidden_size, device=device))

In [12]:
%%timeit -n 1 -r 10
with torch.no_grad():
    for inputs, _ in fake_loader:
        inputs = inputs.to(device)
        model_torch(inputs, hidden)

4.23 s ± 81 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [13]:
%%timeit -n 1 -r 10
with torch.no_grad():
    for inputs, _ in fake_loader:
        inputs = inputs.to(device)
        model_cpp(inputs, hidden)

7.76 s ± 244 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


---

## +Backward time comparison
[go to top](#Peephole-LSTM-Test-&-Performance-Comparison-(Speed-&-Memory))

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
%%timeit -r 10
for inputs, targets in fake_loader:
    inputs = inputs.to(device)
    targets = targets.to(device)
    model_torch.zero_grad()
    criterion(model_torch(inputs, hidden)[0].flatten(0, 1), targets.flatten(0, 1)).backward()

In [ ]:
%%timeit -r 10
for inputs, targets in fake_loader:
    inputs = inputs.to(device)
    targets = targets.to(device)
    model_cpp.zero_grad()
    criterion(model_cpp(inputs, hidden)[0].flatten(0, 1), targets.flatten(0, 1)).backward()

---